# Init

Run this cell to initialise the neccesary variables in the session.

In [ ]:
%run src_ipynb/init.ipynb

# Test Import

Import test from local test database or smartcitizen API:

- Load kits within the selected tests
- Check if there were alphasense sensors and retrieve their calibration data and order of slots
- Check if there was a reference and convert it's units


In [ ]:
%run src_ipynb/load_data.ipynb

## Dataframe cleaning: anomaly detection

Inspired by the code of Dmitriy Sergeev at https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-9-time-series-analysis-in-python-a270cb05e0b3.

### XGBoost regressor

In [ ]:
from src.data.signal_utils import mean_absolute_percentage_error, timeseries_train_test_split, \
                         plotModelResults, prepareDataFrame

test_name = '2018-09_EXT_BOLOGNA_TEST_WALL_MO'
device_name = 'SCK2'

## Ignore columns

irrelevantColumns = ['BATT', 'LIGHT', 'TEMP', 'HUM', 'CO_MICS_THEAT', 'NO2_MICS_THEAT',
       'PRESS', 'GB_1A', 'GB_1W', 'GB_CO', 'GB_2A', 'GB_2W', 'GB_NO2', 'GB_3A',
       'GB_3W', 'GB_O3', 'EXT_TEMP', 'EXT_HUM', 'EXT_PM_1', 'EXT_PM_25',
       'EXT_PM_10']
frequency = '1Min'

# Resample data
data = prepareDataFrame(records.readings[test_name]['devices'][device_name]['data'], 
                 frequency, irrelevantColumns, _numberPasses = 3, _plotModelAnom = True, 
                 _scaleAnom = 1.9, _methodAnom = 'before-after-avg')

# Make a copy to a 'CLEAN' keyword
records.readings[test_name]['devices'][device_name + '_CLEAN'] = dict()

# Put everything except data inside
for key in records.readings[test_name]['devices'][device_name].keys():
    
    if 'data' not in key:
        records.readings[test_name]['devices'][device_name + '_CLEAN'][key] = records.readings[test_name]['devices'][device_name][key]

# Put data inside
records.readings[test_name]['devices'][device_name + '_CLEAN']['data'] = data
records.readings[test_name]['ready_to_model'] = False

# Data Export

In [ ]:
%run src_ipynb/export.ipynb

# Calculator

In [ ]:
%run src_ipynb/calculator.ipynb

# Exploratory Data Analysis

## Time Series Plots

In [ ]:
name = '2019-06_INT_TEST_STATION_21_CSIC'
device = 'STATION_9441'
smooth_list = np.arange(1,30,5)

terms = ['DALLAS_TEMP', 'BATT', 'BATT', 'BATT']
for i in smooth_list:
    formula_name = 'DALLAS_TEMP_SMOOTH_' + str(i)
    formula_value = 'SMOOTH(A, {})'.format(i)
    records.addChannelFormula(name, device, formula_name, terms, formula_value)
    print ("Formula {} Added in test {}, device {}".format(formula_name, name, device))

In [ ]:
%run src_ipynb/ts_plot.ipynb

## Correlation Plot

In [ ]:
%run src_ipynb/corr_plot.ipynb

## Scatter plot matrix

In [ ]:
%run src_ipynb/scatter_plot.ipynb

In [ ]:
%run src_ipynb/scatter_plot.ipynb

## Additional plots

In [ ]:
%run src_ipynb/vizualisation.ipynb

# AlphaSense Baseline Calibration

These functions are used to create the alphasense pollutant correction based on Working, Auxiliary and calibration data provided by alphasense.

In [ ]:
import numpy as np
%matplotlib inline

# ------- INPUT VARIABLE BELOW -------

min_delta = 30
max_delta = 45
overlapHours = 0
prepend_name = 'BASELINE_OVL_AUX'

methods = list()
methods.append(['classic', 'na']) # CO
methods.append(['baseline', 'single_aux']) #NO2
methods.append(['baseline', 'single_aux']) #OX

# ------------------------------------
        
%run src_ipynb/alphasense.ipynb

## Baseline Model Metrics

In [ ]:
## Explore results
# test = '2018-08_INT_STATION_TEST_SUMMER_HOLIDAYS'
test = selectedTestsAD[0]
# device = 'STATION CHIMNEY'
device = '5262'
dataframe = records.readings[test]['devices'][device]['alphasense']['model_stats'][ad_name]

pollutant = 'NO2'

# pollutant_avg 	pollutant_max 	pollutant_min 	pollutant_std 	r2_valueRef 	

fig, (ax, ax2)= plt.subplots(nrows = 2, figsize= (15,10))
ax.plot(dataframe[pollutant]['pollutant_avg'], dataframe[pollutant]['r2_valueRef'], 'o', label=pollutant)
#ax.set_xlim([0,1.5])
ax.set_ylim([0,1])
# plt.setp(ax.xaxis.get_majorticklabels(), rotation=70 )

ax2.bar(dataframe[pollutant].index,dataframe[pollutant]['pollutant_avg'], label=pollutant)
ax2.bar(dataframe[pollutant].index,dataframe[pollutant]['pollutant_max'], label=pollutant, alpha= 0.3)
ax2.bar(dataframe[pollutant].index,dataframe[pollutant]['pollutant_min'], label=pollutant, alpha= 0.3)
ax2.set_ylim([0,100])
#ax.plot(dataframe['O3'].index,dataframe['O3']['pollutant_avg'],'ro', label='O3')
# plt.setp(ax2.xaxis.get_majorticklabels(), rotation=70 )

plt.xlabel('Day')
plt.title('Average pollutant concentration')
plt.legend(loc='best')

## TODO: Correction Checks

In [ ]:
# Sample For stats checks
pollutant = 'NO2'
display(CorrParams[pollutant])

with plt.style.context('seaborn-white'):
    fig1, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(18,8))
            
    ax1.plot(CorrParams[pollutant]['r_valueRef'], CorrParams[pollutant]['avg_temp'], label = 'Temp', linestyle='-', linewidth=0, marker='o')
    ax1.plot(CorrParams[pollutant]['r_valueRef'], CorrParams[pollutant]['avg_hum'] , label = 'Hum', linestyle='-', linewidth=0, marker='o')
    ax2.plot(CorrParams[pollutant]['r_valueRef'], CorrParams[pollutant]['stderr_temp'], label = 'Temp', linestyle='-', linewidth=0, marker='o')
    ax2.plot(CorrParams[pollutant]['r_valueRef'], CorrParams[pollutant]['stderr_hum'] , label = 'Hum', linestyle='-', linewidth=0, marker='o')
    
    ax1.legend(loc='best')
    ax1.set_xlabel('R^2 {} vs Ref'.format(pollutant))
    ax1.set_ylabel('Avg Temp-Hum / day')
    ax1.grid(True)
    ax2.legend(loc='best')
    ax2.set_xlabel('R^2 {} vs Ref'.format(pollutant))
    ax2.set_ylabel('Avg Temp / day')
    ax2.grid(True)
    
    fig2, (ax3, ax4) = plt.subplots(nrows=1, ncols=2, figsize=(18,8))
            
    ax3.plot(CorrParams[pollutant]['r_valueRef'], CorrParams[pollutant]['avg_pollutant'], label = 'Avg Pollutant', linestyle='-', linewidth=0, marker='o')
    ax4.plot(CorrParams[pollutant]['avg_pollutant'], CorrParams[pollutant]['deltaAuxBas_avg'], label = 'Delta', linestyle='-', linewidth=0, marker='o')
    ax4.plot(CorrParams[pollutant]['avg_pollutant'], CorrParams[pollutant]['ratioAuxBas_avg'] , label = 'Ratio', linestyle='-', linewidth=0, marker='o')
    
    ax3.legend(loc='best')
    ax3.set_xlabel('R^2 {} vs Ref'.format(pollutant))
    ax3.set_ylabel('Avg {} / day'.format(pollutant))
    ax3.grid(True)
    ax4.legend(loc='best')
    ax4.set_xlabel('{} Average'.format(pollutant))
    ax4.set_ylabel('Offset / Ratio Baseline vs Auxiliary')
    ax4.grid(True)

# Model Creation

## Batch Model Process (TODO)
Inspired by the example of "Jakob Aungiers, Altum Intelligence ltd" at https://github.com/jaungiers/LSTM-Neural-Network-for-Time-Series-Prediction

In [ ]:
# Linear Regression Utils
from src.models.linear_regression_utils import prep_data_OLS, fit_model_OLS, predict_OLS
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from numpy import concatenate

# ML Utils
from src.models.ml_utils import prep_dataframe_ML, fit_model_ML, predict_ML, get_inverse_transform_ML

# Metrics 
from src.data.signal_utils import metrics
import json

##-------------- Input --------------
configs = json.load(open('./spotcheck_models/models.json', 'r'))
test_model = '2018-08_INT_STATION_TEST_SUMMER_HOLIDAYS'
device_name = 'STATION_CASE'

min_date = '2018-08-01 00:00:00'
max_date = '2018-09-20 00:00:00'

##------------------------------------

for config in configs.keys():
    
    # Retrieve models
    model_name = config
    print model_name

    if model_name not in readings[test_model]['devices'][name_combined_data]['model']:
        model_type = configs[config]['type']
        features = configs[config]['features']
        
        tuple_features = [(k, v) for k, v in features.iteritems()]
        print tuple_features
        
        
        list_features = list()
        for item in tuple_features: 
            if item[0] == 'REF':
                a = tuple_features.index(item)
                list_features.insert(0,item[1] + '_' + device_name)
                reference_name = item[1] + '_' + device_name
            else:
                list_features.append(item[1] + '_' + device_name)
                
        tuple_features[0], tuple_features[a] = tuple_features[a], tuple_features[0]
        
        tuple_features_combined = [(item[0], item[1]  + '_' + device_name) for item in tuple_features]
        
        dataframeModel = readings[test_model]['devices'][name_combined_data]['data'].loc[:,list_features]
        dataframeModel = dataframeModel.dropna()
        
        dataframeModel = dataframeModel[dataframeModel.index > min_date]
        dataframeModel = dataframeModel[dataframeModel.index < max_date]
        
        print '\t{}'. format(model_type) 
        if model_type == 'OLS':
            
            formula_expression = configs[config]['expression']
            ratio_train = configs[config]['ratio_train']
            alpha_filter = configs[config]['alpha_filter']
            n_lags = 1

            ## Model Fit
            dataTrain, dataTest, n_train_periods = prep_data_OLS(dataframeModel, tuple_features_combined, ratio_train, alpha_filter, device_name)
            model = fit_model_OLS(formula_expression, dataTrain, False)
            
            ## Predict the model results
            referenceTrain, predictionTrain = predict_OLS(model, dataTrain, True, False, 'train')
            referenceTest, predictionTest = predict_OLS(model, dataTest, True, False, 'test')

            predictionTrain = predictionTrain.values
            indexTrain = dataTrain['index']
            indexTest = dataTest['index']

        elif model_type == 'LSTM':
            
            epochs = configs[config]['epochs']
            batch_size = configs[config]['batch_size']
            verbose = configs[config]['verbose']
            n_lags = configs[config]['n_lags']
            loss = configs[config]["loss"]
            optimizer = configs[config]["optimizer"]
            layers = configs[config]['layers']
            ratio_train = configs[config]['ratio_train']

            ## Prep Dataframe
            index, train_X, train_y, test_X, test_y, scalerX, scalery, n_train_periods = prep_dataframe_ML(dataframeModel, min_date, max_date, list_features, n_lags, ratio_train, alpha_filter, reference_name, False)
            ## Model Fit
            model = fit_model_ML(train_X, train_y, 
                                 test_X, test_y, 
                                 epochs = epochs, 
                                 batch_size = batch_size, 
                                 verbose = True, 
                                 plotResult = False, 
                                 loss = loss, 
                                 optimizer = optimizer,
                                 layers = layers)
            
            # Get model prediction
            
            # Get model prediction
            referenceTrain = get_inverse_transform_ML(train_y, n_lags, scalery)
            predictionTrain = predict_ML(model, train_X, n_lags, scalery)

            referenceTest = get_inverse_transform_ML(test_y, n_lags, scalery)
            predictionTest = predict_ML(model, test_X, n_lags, scalery)
            
            indexTrain = index[:n_train_periods]
            indexTest = index[n_train_periods+n_lags:]
            formula_expression = '-'
        
    
        dataFrameTrain = pd.DataFrame(data = {'reference': referenceTrain, 'prediction': predictionTrain}, 
                                      index = indexTrain)
        dataFrameTest = pd.DataFrame(data = {'reference': referenceTest, 'prediction': predictionTest}, 
                                      index = indexTest)
        
        dataFrameExport = dataFrameTrain.copy()
        dataFrameExport = dataFrameExport.combine_first(dataFrameTest)

        # Get model metrics
        metrics_model_train = metrics(referenceTrain, predictionTrain)
        metrics_model_test = metrics(referenceTest, predictionTest)
        
        ## Put everything in the dict
        dictModel = readings[test_model]['devices'][name_combined_data]
        
        # From https://hackmd.io/Y62wiJw0RaiBfU4Xhv8dQQ#
        dictModel[model_name] = dict()
        dictModel[model_name]['metrics'] = dict()
        dictModel[model_name]['metrics']['train'] = metrics_model_train
        dictModel[model_name]['metrics']['test'] = metrics_model_test
        
        # Model Parameters
        dictModel[model_name]['parameters'] = dict()
        dictModel[model_name]['parameters']['features'] = dict()
        dictModel[model_name]['parameters']['features']['ref'] = tuple_features[0]
        dictModel[model_name]['parameters']['features']['items'] = tuple_features[1:]        
        dictModel[model_name]['parameters']['ratio_train'] = ratio_train
        dictModel[model_name]['parameters']['alpha_filter'] = alpha_filter
        dictModel[model_name]['model'] = model
        dictModel[model_name]['modelType'] = model_type
        
        # Put it back in the readings dataframe
        readings[test_model]['devices'][name_combined_data]['model'][model_name] = dictModel[model_name]
        readings[test_model]['devices'][model_name] = dict()
        readings[test_model]['devices'][model_name]['data'] = dataFrameExport
        print '\t Model Calculated'
        
    else:
        print '\t Model already present, skipping'

print 'All models calculated'

## Ordinary Linear Regression

### Preliminary Checks



#### Data stationarity

We will use the Dicker-fuller test (ADF) test to verify the data is stationary. We need to check data stationarity for certain type of models. 

If the process is stationary means it doesn’t change its statistical properties over time: mean and variance do not change over time (constancy of variance is also called homoscedasticity), also covariance function does not depend on the time (should only depend on the distance between observations) Source [here](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-9-time-series-analysis-in-python-a270cb05e0b3). Visually:

- Growing mean --> Non stationary

![](https://cdn-images-1.medium.com/max/800/0*qrYiVksz8g3drl5Z.png)

- Growing spread --> Non stationary

![](https://cdn-images-1.medium.com/max/800/0*fEqQDq_TaEqa511n.png)

- Varying time covariance --> Non stationary

![](https://cdn-images-1.medium.com/max/800/1*qJs3g2f77flIXr6mFsbPmw.png)

- Null Hypothesis (H0): If failed to be rejected, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.
- Alternate Hypothesis (H1): The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary. It does not have time-dependent structure.

We interpret this result using the p-value from the test. A p-value below a threshold (such as 5% or 1%) suggests we reject the null hypothesis (stationary), otherwise a p-value above the threshold suggests we fail to reject the null hypothesis (non-stationary). Hence:

- p-value > 0.05: fail to reject the null hypothesis (H0), the data has an unit root and is non-stationary.
- p-value <= 0.05: reject the null hypothesis (H0), the data does not have an unit root and is stationary.

#### Autocorrelation

High levels of autocorrelation can indicate our time series is shows seasonality. We will use the ACF plot to identify possible autocorrelation and potentially include differentiation.

#### Granger Casuality Test (use with caution)

This test is useful to determine the casuality of variables and determining whether one time series is useful in forecasting another. 

The Null hypothesis for granger causality tests is that the time series in the second column, x2, does NOT Granger cause the time series in the first column, x1. Grange causality means that past values of x2 have a statistically significant effect on the current value of x1, taking past values of x1 into account as regressors. We reject the null hypothesis that x2 does not Granger cause x1 if the pvalues are below a desired size of the test. Hence:

- p-value < size: allows to reject the null hypothesis (H0) for x1 = f(x2)
- p-value > size: we fail to reject the null hypothesis (H0) for x1 = f(x2)

The null hypothesis for all four test is that the coefficients corresponding to past values of the second time series are zero.

Reference [here](https://en.wikipedia.org/wiki/Granger_causality), [here](https://stats.stackexchange.com/questions/24753/interpreting-granger-causality-tests-results#24796) and [here](http://www.statsmodels.org/devel/generated/statsmodels.tsa.stattools.grangercausalitytests.html).

### Model fit

In [ ]:
from src.models.linear_regression_utils import prep_data_OLS, fit_model_OLS, predict_OLS, plot_OLS_coeffs
from src.models.linear_regression_utils import tfuller_plot
from statsmodels.tsa.stattools import grangercausalitytests

test_model = '2018-08_INT_STATION_TEST_SUMMER_HOLIDAYS'
tuple_features = (['REF', 'EXT_PM_1', 'STATION_CASE'],
                 ['A', 'EXT_PM_10', 'STATION_CASE'],
                 ['B', 'EXT_TEMP', 'STATION_CASE'])#,
                 #['C', 'PM_DALLAS_TEMP_DERIV', '5262_processed'])

min_date = '2018-01-01 00:00:00'
max_date = '2019-04-20 00:00:00'

model_name = 'TEST_MODEL'
model_target = 'ALPHASENSE' # ALPHASENSE, PMS, MICS...
model_type = 'OLS'
formula_expression = 'REF ~ A + np.log(B)'#' + C'
alpha_filter = None #No filtering
ratio_train = 3./4 # Important that this is a float, don't forget the .
model_full_name = '_'.join([model_target, model_type, model_name])
print ('Model Name', model_full_name)

print ('Preparing devices from test {}'.format(test_model))
records.prepare_dataframe_model(tuple_features, test_model, None, None, 
                                      model_full_name, clean_na = True, clean_na_method = 'fill' , 
                                      target_raster = '5Min')

dataframeModel = records.readings[test_model]['models'][model_full_name]['data']
reference_name = records.readings[test_model]['models'][model_full_name]['reference']
labels = dataframeModel[reference_name]
features = dataframeModel.drop(reference_name, axis = 1)

# List of features for later use
feature_list = list(features.columns)
n_features = len(tuple_features)-1

## Preliminary Check
# Fuller Plot for all
# for column in dataframeModel:
#     x = dataframeModel.loc[:,column]
#     tfuller_plot(x, name = x.name, lags=60, lags_diff = 5)
# Granger Causality
# print ('--------------------------------------')
# print ('Granger Causality Test')
## Granger Causality Test (WIP)
# for item in tuple_features:
#     if item[0] != 'REF':
#         print '\nCausality for x1 = {} and x2 = {}'.format(reference_name, item[1])
#         x = dataframeModel.loc[:,[reference_name, item[1]]].dropna()
#         x = x.values
#         granger_causality_tests = grangercausalitytests(x, 1)
#         # print granger_causality_tests

## Prepare Dataframe
dataTrain, dataTest, n_train_periods = prep_data_OLS(dataframeModel, 
                                                     tuple_features, 
                                                     ratio_train)

## Model Fit
model = fit_model_OLS(formula_expression, dataTrain, printSummary = True)
plot_OLS_coeffs(model)

### Model prediction and archiving

In [ ]:
from src.models.linear_regression_utils import predict_OLS
from src.data.signal_utils import metrics
import pandas as pd

## Predict the model results
dataFrameTrain = predict_OLS(model, dataTrain, True, False, 'train')
dataFrameTest = predict_OLS(model, dataTest, True, False, 'test')

## Combine them for export
dataFrameExport = dataFrameTrain.copy()
dataFrameExport = dataFrameExport.combine_first(dataFrameTest)

# Get Metrics
metrics_model = dict()
metrics_model['train'] = metrics(dataFrameTrain['reference'], dataFrameTrain['prediction'])
metrics_model['test'] = metrics(dataFrameTest['reference'], dataFrameTest['prediction'])

records.archive_model(test_model, model_full_name, 
                      metrics_model, 
                      dataFrameExport, model, model_type, 
                      model_target, ratio_train, formula_expression, alpha_filter = alpha_filter)

print ('Metrics Summary:')
print ("{:<23} {:<7} {:<5}".format('Metric','Train','Test'))
for metric in metrics_model['train'].keys():
    print ("{:<20}".format(metric) +"\t" +"{:0.3f}".format(metrics_model['train'][metric]) +"\t"+ "{:0.3f}".format(metrics_model['test'][metric]))

### Diagnostics plots

In [ ]:
from src.models.linear_regression_utils import model_R_plots
%matplotlib inline

model_R_plots(model, dataTrain, dataTest)

## LSTM

### Model Fit

In [ ]:
from src.models.ml_utils import prep_dataframe_ML, fit_model_ML, predict_ML, get_inverse_transform_ML
from src.data.signal_utils import metrics

test_model = '2018-08_INT_STATION_TEST_SUMMER_HOLIDAYS'
tuple_features = (['REF', 'GB_1W', 'STATION_CASE'],
                 ['A', 'CO_MICS_RAW', 'STATION_CASE'],
                 ['B', 'TEMP', 'STATION_CASE'],
                 ['C', 'HUM', 'STATION_CASE'],
                 ['D', 'PM_25', 'STATION_CASE'])

min_date = '2018-08-01 00:00:00'
max_date = '2018-09-20 00:00:00'

model_name = 'TEST_MODEL'
model_target = 'MICS' # ALPHASENSE, PMS, MICS...
model_type = 'LSTM' # 
ratio_train = 3./4 # Important that this is a float, don't forget the .
n_lags = 2
model_full_name = '_'.join([model_target, model_type, model_name, 'n_lags-{}'.format(n_lags)])
print ('Model Name', model_full_name)

# Number of lags for the model
n_lags = 4

print ('Preparing devices from test {}'.format(test_model))
records.prepare_dataframe_model(tuple_features, test_model, min_date, max_date, 
                                      model_full_name, clean_na = True, clean_na_method = 'drop' , 
                                      target_raster = '10Min')

dataframeModel = records.readings[test_model]['models'][model_full_name]['data']
reference_name = records.readings[test_model]['models'][model_full_name]['reference']

labels = dataframeModel[reference_name]
features = dataframeModel.drop(reference_name, axis = 1)

# List of features for later use
feature_list = list(features.columns)
n_features = len(tuple_features)-1

# Data Split
train_X, train_y, test_X, test_y, scalerX, scalery = prep_dataframe_ML(dataframeModel, 
                                                                       n_features, 
                                                                       n_lags, ratio_train)

index = dataframeModel.index
n_train_periods = train_X.shape[0]

# Model Fit
print ('Model training...')
model = fit_model_ML('LSTM', train_X, train_y, 
                       test_X, test_y, 
                       epochs = 50, batch_size = 72, 
                       verbose = 2, plotResult = True, 
                       loss = 'mse', optimizer = 'adam')

### Model prediction and archiving

In [ ]:
from src.models.ml_utils import predict_ML, get_inverse_transform_ML
from src.data.signal_utils import metrics
import matplotlib.pyplot as plot
%matplotlib inline

# Get model prediction
inv_train_y = get_inverse_transform_ML(train_y, scalery)
dataFrameTrain = predict_ML(model, train_X, 
                            inv_train_y, index[:n_train_periods],
                            scalery)

inv_test_y = get_inverse_transform_ML(test_y, scalery)
dataFrameTest = predict_ML(model, test_X, 
                           inv_test_y, index[n_train_periods+n_lags:], 
                           scalery)

dataFrameExport = dataFrameTrain.copy()
dataFrameExport = dataFrameExport.combine_first(dataFrameTest)

# Get Metrics
print ('Calculating Metrics...')
metrics_model = dict()
metrics_model['train'] = metrics(dataFrameTrain['reference'], dataFrameTrain['prediction'])
metrics_model['test'] = metrics(dataFrameTest['reference'], dataFrameTest['prediction'])

print ('Archiving Model...')
records.archive_model(test_model, model_full_name, 
                      metrics_model, 
                      dataFrameExport, model, model_type, 
                      model_target, ratio_train, n_lags = n_lags, scalerX = scalerX, scalery = scalery)

print ('Metrics Summary:')
print ("{:<23} {:<7} {:<5}".format('Metric','Train','Test'))
for metric in metrics_model['train'].keys():
    print ("{:<20}".format(metric) +"\t" +"{:0.3f}".format(metrics_model['train'][metric]) +"\t"+ "{:0.3f}".format(metrics_model['test'][metric]))

## Random Forest Regressor or SVR

No feature scaling implementation

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from src.data.signal_utils import metrics
import matplotlib.pyplot as plot
import numpy as np
from sklearn.model_selection import train_test_split


###-------------------------------------------INPUT INFO HERE-----------------------------------------------------
test_model = '2018-08_INT_STATION_TEST_SUMMER_HOLIDAYS'
tuple_features = (['REF', 'EXT_PM_1', 'STATION_CASE'],
                 ['A', 'EXT_PM_10', 'STATION_CASE'],
                 ['B', 'EXT_TEMP', 'STATION_CASE'])#,
                 #['C', 'PM_DALLAS_TEMP_DERIV', '5262_processed'])

min_date = '2018-01-01 00:00:00'
max_date = '2019-03-20 00:00:00'

model_name = 'NO2_BASE-T-dT_Shuffle-Dublin_3-4_5Min'
model_target = 'ALPHASENSE' # ALPHASENSE, PMS, MICS...
shuffle_split = True
model_type = 'RF' # RF or SVR
ratio_train = 3./4 # Important that this is a float, don't forget the .

###----------------------------------------------------------------------------------------------------------------
model_full_name = '_'.join([model_target, model_type, model_name])
print ('Model Name', model_full_name)

print ('Preparing devices from test {}'.format(test_model))
records.prepare_dataframe_model(tuple_features, test_model, min_date, max_date, 
                                      model_full_name, clean_na = True, clean_na_method = 'drop' , 
                                      target_raster = '5Min')

dataframeModel = records.readings[test_model]['models'][model_full_name]['data']
reference_name = records.readings[test_model]['models'][model_full_name]['reference']

labels = dataframeModel[reference_name]
features = dataframeModel.drop(reference_name, axis = 1)

# List of features for later use
feature_list = list(features.columns)

features = np.array(features)
labels = np.array(labels)
# Training and Testing Sets
train_X, test_X, train_y, test_y = train_test_split(features, labels, random_state = 42, 
                                                    test_size = 1-ratio_train, shuffle = shuffle_split)

n_train_periods = train_X.shape[0]
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

### Single Model Fit - Prediction and Archiving

In [ ]:
from src.models.ml_utils import predict_ML

# Instantiate model 
if model_type == 'RF':
    model = RandomForestRegressor(n_estimators= 1000, random_state = 42)
elif model_type == 'SVR':
    model = SVR(kernel='rbf')
    
## Train the model on training data
print ('Training Model {}...'.format(model_name))
model.fit(train_X, train_y)

## Get model prediction
dataFrameTrain = predict_ML(model, features[:n_train_periods], labels[:n_train_periods], dataframeModel.index[:n_train_periods])
dataFrameTest = predict_ML(model, features[n_train_periods:], labels[n_train_periods:], dataframeModel.index[n_train_periods:])

# Get model metrics
print ('Calculating Metrics...')
metrics_model = dict()
metrics_model['train'] = metrics(dataFrameTrain['reference'], dataFrameTrain['prediction'])
metrics_model['test'] = metrics(dataFrameTest['reference'], dataFrameTest['prediction'])

dataFrameExport = dataFrameTrain.copy()
dataFrameExport = dataFrameExport.combine_first(dataFrameTest)
print ('Archiving Model...')
records.archive_model(test_model, model_full_name, 
                      metrics_model, 
                      dataFrameExport, model, model_type, 
                      model_target, ratio_train, shuffle_split)

print ('Metrics Summary:')
print ("{:<23} {:<7} {:<5}".format('Metric','Train','Test'))
for metric in metrics_model['train'].keys():
    print ("{:<20}".format(metric) +"\t" +"{:0.3f}".format(metrics_model['train'][metric]) +"\t"+ "{:0.3f}".format(metrics_model['test'][metric]))

### Random Search with Cross Validation 

We perform here cross validated random search of the model hyperparameters, to later on retrieve the best parameters with a grid search around the best found results of the CV.

Using **k-fold cross validation** below:

![](https://i.imgur.com/HLbgMSS.png)

Source: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from src.models.ml_utils import predict_ML

if model_type == 'RF':

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    ## Evaluate the default model
    base_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    random_model = RandomizedSearchCV(estimator=RandomForestRegressor(), param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1)
elif model_type == 'SVR':
    
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    random_grid = {"C": [1e0, 1e1, 1e2, 1e3], 
                   "gamma": np.logspace(-2, 2, 5),
                   "kernel": ['rbf', 'sigmoid'],
                  "shrinking": [True, False]}
    
    ## Create the default model
    base_model = SVR(kernel='rbf', gamma=0.1)

    ## Create randomized Search
    random_model = RandomizedSearchCV(estimator = SVR(), cv=5, 
                             n_iter = 100, scoring = 'neg_mean_absolute_error',
                             param_distributions=random_grid,  verbose=2, random_state=42, n_jobs=-1)
    
# Fit the base model
base_model.fit(train_X, train_y)
## Get base model prediction
dataFrameTrain_base = predict_ML(base_model, features[:n_train_periods], labels[:n_train_periods], dataframeModel.index[:n_train_periods])
dataFrameTest_base = predict_ML(base_model, features[n_train_periods:], labels[n_train_periods:], dataframeModel.index[n_train_periods:])

# Fit the random search model
random_model.fit(train_X, train_y)
random_model.best_params_
best_random = random_model.best_estimator_
## Evaluate the best model
dataFrameTrain_best = predict_ML(best_random, features[:n_train_periods], labels[:n_train_periods], dataframeModel.index[:n_train_periods])
dataFrameTest_best = predict_ML(best_random, features[n_train_periods:], labels[n_train_periods:], dataframeModel.index[n_train_periods:])

Now perform the gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

if model_type == 'RF':
    # Create the parameter grid based on the results of random search 
    param_grid = {
        'bootstrap': [False],
        'max_depth': [80, 90, 100, 110],
        'max_features': [2, 3],
        'min_samples_leaf': [1, 2],
        'min_samples_split': [2, 3],
        'n_estimators': [200, 300, 400, 1000]
    }
        
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_grid, 
                               scoring = 'neg_mean_absolute_error', cv = 3, 
                               n_jobs = -1, verbose = 2)
elif model_type == 'SVR':

    # Create the parameter grid based on the results of random search 
    param_grid = {"C": [1e0, 1e1, 1e2, 1e3], 
                   "gamma": np.logspace(-2, 2, 5),
                  "shrinking": [True, False]
    }

    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = SVR(), param_grid = param_grid, 
                               scoring = 'neg_mean_absolute_error', cv = 3, 
                               n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(train_X, train_y)
# Fit the grid search to the data
grid_search.fit(train_X, train_y)

Get best grid estimator

In [ ]:
print (grid_search.best_params_)
best_grid = grid_search.best_estimator_
print (best_grid)
dataFrameTrain_best_grid = predict_ML(best_grid, features[:n_train_periods], labels[:n_train_periods], dataframeModel.index[:n_train_periods])
dataFrameTest_best_grid = predict_ML(best_grid, features[n_train_periods:], labels[n_train_periods:], dataframeModel.index[n_train_periods:])

If happy with the best predictions of the grid search, put them in the dataframe for plotting and archiving

In [ ]:
dataFrameExport = dataFrameTrain_best_grid.copy()
dataFrameExport = dataFrameExport.combine_first(dataFrameTest_best_grid)

# Get model metrics
metrics_model = dict()
metrics_model['train'] = metrics(dataFrameTrain_best_grid['reference'], dataFrameTrain_best_grid['prediction'])
metrics_model['test'] = metrics(dataFrameTest_best_grid['reference'], dataFrameTest_best_grid['prediction'])

records.archive_model(test_model, model_full_name + '_best_grid_search', 
                      metrics_model, 
                      dataFrameExport, best_grid, model_type, 
                      model_target, ratio_train)

print ('Metrics Summary:')
print ("{:<23} {:<7} {:<5}".format('Metric','Train','Test'))
for metric in metrics_model['train'].keys():
    print ("{:<20}".format(metric) +"\t" +"{:0.3f}".format(metrics_model['train'][metric]) +"\t"+ "{:0.3f}".format(metrics_model['test'][metric]))

## Model plots

In [ ]:
from src.models.ml_utils import plot_model_ML
%matplotlib inline

plot_model_ML(model, dataFrameTrain, dataFrameTest, feature_list, model_type, model_name)

## Model Export to Disk

In [ ]:
out_model_export = widgets.Output()

selectedModel = tuple()
def selectModel(Models):
    global selectedModel
    selectedModel = list(Models)
    # selectedTestBases = list()
    # selectedTestBases.append('')
    # for test in selectedTest:
    #     selectedTestBases.append(basename(normpath(test)))
    # name_drop_api.options = selectedTestBases
    
def exportModel(b):
    with out_model_export:
        clear_output()
        for model_selected in selectedModel:
            print ('Exporting model', model_selected, '...')
            records.export_model(test_model, model_selected)
            print ('---')
        
button_model_export = widgets.Button(description='Export Selected')
button_model_export.on_click(exportModel)

interact(selectModel,
    Models = widgets.SelectMultiple(options=records.readings[test_model]['models'].keys(), 
                            selected_labels = selectedModel,
                            layout=widgets.Layout(width='700px')))

# records.export_model(test_model, model_full_name, modelDirectory)
display(button_model_export)
display(out_model_export)

## Model Comparison

### TimeSeries Comparison

In [ ]:
import matplotlib.pyplot as plt
import plotly.tools as tls
%matplotlib inline

fig = plt.figure(figsize=(15,10))
referencePlotted = False
        
for model_name in records.readings[test_model]['models']:
    try:
        ratio_train = records.readings[test_model]['models'][model_name]['parameters']['ratio_train']
        data = records.readings[test_model]['devices'][model_name]['data']
                
        total_len = len(data.index)
        n_train_periods = int(round(total_len*ratio_train))
    
        dataframeTrain = data.iloc[:n_train_periods,:]
        dataframeTest = data.iloc[n_train_periods:,:]
                        
        if (not referencePlotted):
            plt.plot(dataframeTrain.index, dataframeTrain['reference'], 'b.', label = 'Reference Train', alpha = 0.3)
            plt.plot(dataframeTest.index, dataframeTest['reference'], 'b.', label = 'Reference Test', alpha = 0.3)
            referencePlotted = True
            
        plt.plot(dataframeTrain.index, dataframeTrain['prediction'], linewidth = 0.6, label = 'Prediction Train ' + model_name)
        plt.plot(dataframeTest.index, dataframeTest['prediction'], linewidth = 0.6, label = 'Prediction Test ' + model_name)
    except:
        pass
plt.legend(loc = 'best')
# plt.ylabel(str(readings[test_model]['devices'][name_combined_data]['model'][model_name]['parameters']['features']["ref"][1]))
plt.xlabel('Date (-)')
plt.grid(True)
# plt.title('Model Comparison for ' + str(readings[test_model]['devices'][name_combined_data]['model'][model_name]['parameters']['features']["ref"][1]))

### Scatter Comparison

In [ ]:
import matplotlib.pyplot as plot
from matplotlib import gridspec
import pandas as pd
import math
%matplotlib inline

number_of_subplots = len(records.readings[test_model]['models'].keys()) 
if number_of_subplots % 2 == 0: cols = 2
else: cols = 3
rows = int(math.ceil(number_of_subplots / cols))
gs = gridspec.GridSpec(rows, cols)
fig = plot.figure(figsize=(15,10))

fig.tight_layout()
n = 0

for model_name in records.readings[test_model]['models']:
    try:
        ratio_train = records.readings[test_model]['models'][model_name]['parameters']['ratio_train']
    
        data = records.readings[test_model]['devices'][model_name]['data']
        dataVal = data.groupby(pd.Grouper(freq='1H')).aggregate(np.mean)    
        total_len = len(dataVal.index)
        n_train_periods = int(round(total_len*ratio_train))
        
        dataframeTrain = dataVal.iloc[:n_train_periods,:]
        dataframeTest = dataVal.iloc[n_train_periods:,:]
    
        ax = fig.add_subplot(gs[n])
        n += 1          
        plot.plot(dataframeTrain['reference'], dataframeTrain['prediction'], 'go', label = 'Train ' + model_name, alpha = 0.3)
        plot.plot(dataframeTest['reference'], dataframeTest['prediction'], 'bo', label = 'Test ' + model_name, alpha = 0.3)
        plot.plot(dataframeTrain['reference'], dataframeTrain['reference'], 'k', label = '1:1 Line', linewidth = 0.4, alpha = 0.3)
     
        plot.legend(loc = 'best', prop={'size': 8})
        plot.ylabel('Prediction (-)')
        plot.xlabel('Reference (-)')
        plot.show()
    except:
        print ("Model {} was not archived properly".format(model_name))
        pass

In [ ]:
import matplotlib.pyplot as plot
from matplotlib import gridspec
import math
from src.data.signal_utils import metrics

number_of_subplots = 3
if number_of_subplots % 2 == 0: cols = 2
else: cols = 3
rows = int(math.ceil(number_of_subplots / cols))
gs = gridspec.GridSpec(rows, cols)
fig = plot.figure(figsize=(20,6))

fig.tight_layout()
n = 0
test = '2018-09_EXT_BOLOGNA_TEST_WALL_MO'

pollutant_list = ['CO_AD_BASE1-60', 'NO2_AD_BASE1-60', 'O3_AD_BASE1-60']
ref_list = ['CO_REF', 'NO2_REF', 'O3_REF']
min_date = '2018-08-21 14:45:00'
max_date = '2018-08-28 23:45:00'

for i in range(3):

    data_source_1 = records.readings[test]['devices']['SCK2']['data']
    # data_source_2 = records.readings[test]['devices']['5528']['data']
    data_ref = records.readings[test]['devices']['ARPAE_MO']['data']
    
    data_source_1_val = data_source_1.groupby(pd.Grouper(freq='1H')).aggregate(np.mean)
    data_ref_val = data_ref.groupby(pd.Grouper(freq='1H')).aggregate(np.mean)
    
    data_source_1_val = data_source_1_val[data_source_1_val.index > min_date]
    data_source_1_val = data_source_1_val[data_source_1_val.index < max_date]

    data_ref_val = data_ref_val[data_ref_val.index > min_date]
    data_ref_val = data_ref_val[data_ref_val.index < max_date]
    
    ax = fig.add_subplot(gs[n])
    n += 1          
    plot.plot(data_ref_val[ref_list[i]], data_source_1_val[pollutant_list[i]], 'go', label = 'SC Station #5527', alpha = 0.5)
    # plot.plot(data_ref[ref_list[i]], data_source_2[pollutant_list[i]], 'bo', label = 'SC Station #5528', alpha = 0.3)
    plot.plot(data_ref_val[ref_list[i]], data_ref_val[ref_list[i]], 'k', label = '1:1 Line', linewidth = 0.4, alpha = 0.3)
 
    plot.legend(loc = 'best')
    plot.ylabel('Pollutant {}'.format(ref_list[i]))
    plot.xlabel('Pollutant {}'.format(ref_list[i]))
    plot.title('Hola')
    
    metrics_poll = metrics(data_ref_val[ref_list[i]], data_source_1_val[pollutant_list[i]])
    print (metrics_poll)
    

### Model Metrics Comparison: target diagram

In [ ]:
from src.visualization.visualization import targetDiagram
%matplotlib inline

for model in records.readings[test_model]['models']:
    try:
        print ('-----------------------------------------------------')
        print ('\nModel Name: {}'.format(model))
        print ("{:<23} {:<7} {:<5}".format('Metric','Train','Test'))
        metrics_model = records.readings[test_model]['models'][model]['metrics']
        for metric in metrics_model['train'].keys():
            print ("{:<20}".format(metric) +"\t" +"{:0.3f}".format(metrics_model['train'][metric]) +"\t"+ "{:0.3f}".format(metrics_model['test'][metric]))
    except:
        print ('Cannot use model {}'.format(model))
targetDiagram(records.readings[test_model]['models'], True)

## Model Evaluation

### Data Quality Objectives (TO CHECK)
Explained here http://dx.doi.org/10.1016/j.envint.2016.12.007

Sensor values Y, reference values x

In [ ]:
from scipy.stats.stats import linregress
import matplotlib.pyplot as plot
import numpy as np
%matplotlib inline

def fUEREL(ux, values_x, values_Y):
    def RSS(values_x, values_Y, intercept, slope):
        pre_sum_1 = np.power(values_Y - intercept - np.multiply(slope, values_x), 2)
        # pre_sum_2 = np.power(values_Y / (intercept + np.multiply(slope, values_x)) - 1, 2)
        
        # fig, axes = plot.subplots(1, 2, figsize=(15,10))
        # axes[0].plot(pre_sum_1)
        # axes[1].plot(pre_sum_2)

        RSS = np.sum(np.power(pre_sum_1,2))
        
        return RSS
    
    slope, intercept, _, _, _ = linregress(values_x, values_Y)
    # fig = plot.figure(figsize=(15,10))
    # plot.plot(slope*values_x + intercept, label='Sensor')
    # plot.plot(values_Y)
    
    RSS = RSS(values_x, values_Y, intercept, slope)
    n = len(values_x)
    if len(values_Y) != n: return
    A = RSS/(n-2)-np.power(ux,2)
    B = np.power(intercept + (slope-1)*values_x, 2)
    C = np.power(A + B, 0.5)
    UEREL = np.divide(2*C, values_Y)
    
    return UEREL

In [ ]:
import matplotlib.pyplot as plot
from matplotlib import gridspec
import math

dqo_table = (['PM', 50],
            ['O3', 30],
            ['CO',25],
            ['NO',25],
            ['NO2',25],
            ['NOX',25],
            ['SO2',25])

ux = 0
test_model = '2019-02_EXT_DUBLIN_URBAN_BACKGROUND'

## ---
number_of_subplots = len(records.readings[test_model]['models'].keys()) 

if number_of_subplots % 2 == 0: cols = 2
else: cols = 3
    
rows = int(math.ceil(number_of_subplots / cols))
gs = gridspec.GridSpec(rows, cols)
fig = plot.figure(figsize=(15,10))

fig.tight_layout()
n = 0

for model in records.readings[test_model]['models']:
    try:
        data = records.readings[test_model]['devices'][model]['data']
        dataVal = data.groupby(pd.Grouper(freq='1H')).aggregate(np.mean)
        values_x = dataVal['reference'].values
        values_Y = dataVal['prediction'].values
        
        total_len = len(data.index)
        n_train_periods = int(round(total_len*ratio_train))
    
        ax = fig.add_subplot(gs[n])
        n += 1      
    
        uerel = 100*fUEREL(ux, values_x, values_Y)
        
        plot.plot(values_x, uerel, 'ko')
        plot.xlabel('Ref. conc [ppb]')
        plot.ylabel('Rel. Exp. Unc (%)')
        plot.ylim([0, 100])
        plot.title(model)
        plot.axhline(y=25, color='r', linestyle='-')
    except:
        pass
            

# Model Application

In [ ]:
%run src_ipynb/apply_model.ipynb

In [ ]:
# from os.path import join
from sklearn.externals import joblib
from keras.models import model_from_json
import json
from IPython.display import display
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

out_model_load = widgets.Output()

dict_models = dict()
with open(join(modelDirectory, 'summary.json'), 'r') as summary_file:
    dict_models = json.load(summary_file)
    
selectedModels = tuple()
def load_selectModels(selected_model):
    with out_model_load:
        global selectedModels
        selectedModels = list(selected_model)
        print (selectedModels)
    
def load_show_devices(Source):
    load_device.options = [s for s in list(records.readings[Source]['devices'].keys())]
    load_device.source = Source
    
def load_show_dates(Source):
    load_min_date.value = records.readings[Source]['devices'][load_device.value]['data'].index.min()._short_repr
    load_max_date.value = records.readings[Source]['devices'][load_device.value]['data'].index.max()._short_repr

def loadModel(b):
    with out_model_load:
        clear_output()
            
        if len(selectedModels)>0:
            global loaded_model
            global loaded_params
            global loaded_metrics
            global loaded_features
            global loaded_model_name
            
            if 'ARCHIVE' in selectedModels[0]:
                model_name = selectedModels[0][8:]
                print ('Loading model {} from disk'.format(model_name))
                filename = join(modelDirectory, load_target_drop.value, model_name)
                if load_type_drop.value == "LSTM":
                    # ML Model
                    # Load Model and weights
                    json_file = open(filename + "_model.json", "r")
                    loaded_model_json = json_file.read()
                    json_file.close()
                
                    loaded_model = model_from_json(loaded_model_json)
                    loaded_model.load_weights(filename + "_model.h5")
                elif load_type_drop.value == "OLS" or load_type_drop.value == 'RF' or load_type_drop.value == 'SVR':
                    # OLS, RF, or SVR Model
                    loaded_model = joblib.load(filename + '_model.sav')
                    
                # Load params and metrics
                loaded_params = joblib.load(filename + '_parameters.sav')
                loaded_metrics = joblib.load(filename + '_metrics.sav')
                loaded_features = joblib.load(filename + '_features.sav')
                print ('Model loaded from disk')
            elif 'SESSION' in selectedModels[0]:
                model_name = selectedModels[0][8:]

                test_source = list_tests[list_model_session.index(selectedModels[0])]
                print ('Using model {} from current session'.format(model_name))

                loaded_model = records.readings[test_source]['models'][model_name]['model']
                loaded_params = records.readings[test_source]['models'][model_name]['parameters']
                loaded_metrics = records.readings[test_source]['models'][model_name]['metrics']
                loaded_features = records.readings[test_source]['models'][model_name]['features']
                loaded_ref = records.readings[test_source]['models'][model_name]['reference']
                print ('Model loaded from session')
            display(Markdown('## Model Load'))
            display(Markdown("Loaded " + model_name))
            display(Markdown('**Model Type** (*loaded_model*):' ))
            display(loaded_model)
            display(Markdown('**Model Parameters** (*loaded_params*)'))
            display(loaded_params)
            display(Markdown('**Model Metrics** (*loaded_metrics*)'))
            display(loaded_metrics)
            display(Markdown('**Model Features** (*loaded_features*)'))
            display(loaded_features)
            loaded_model_name = model_name
        else:
            print ('Select one model to load')
    
def load_show_models(target, mtype):
    with out_model_load:
        clear_output()
        global list_tests
        global list_model_session
        list_models = list()
        for item in dict_models[target]:
            if dict_models[target][item] == mtype:
                list_models.append('ARCHIVE_' + item)
        list_tests = list()
        list_model_session = list()
        for reading in records.readings:
            if 'models' in records.readings[reading]:
                for model_name in records.readings[reading]['models']:
                    try:
                        if records.readings[reading]['models'][model_name]['model_type'] == mtype:
                            list_models.append('SESSION_' + model_name)
                            list_tests.append(reading)
                            list_model_session.append('SESSION_' + model_name)
                    except:
                        print ('Could not use model {} from current session. Model is not archived'.format(model_name))
        load_models.options = list(list_models)
        
def load_calculateModel(b):
    with out_model_load:
        
        load_test_name = load_device.source
        load_device_name = load_device.value
        load_prediction_name = load_result_text.value
        
        clear_output()
        # Predict based on choices
        records.predict_channels(load_test_name, load_device_name, loaded_model, loaded_features, loaded_params, 
                         load_type_drop.value, loaded_model_name, load_result_text.value, plot_result.value, load_min_date.value, load_max_date.value, 
                         clean_na = True, clean_na_method = 'fill', target_raster = '1Min')

display(widgets.HTML('<hr><h4>Import Local Models</h4>'))

# Test dropdown
load_test = widgets.Dropdown(options=[k for k in records.readings.keys()], 
                        layout=widgets.Layout(width='400px'),
                        description = 'Test')

load_test_drop = widgets.interactive(load_show_devices, 
                                Source=load_test, 
                                layout=widgets.Layout(width='600px'))

load_type_drop = widgets.Dropdown(options = ['LSTM', 'RF', 'OLS', 'SVR'],
                                  value = 'LSTM',
                                  description = 'Model Type',
                                  layout = widgets.Layout(width='300px'))

load_target_drop = widgets.Dropdown(options = ['ALPHASENSE', 'MICS', 'PMS'],
                                  value = 'MICS',
                                  description = 'Model Target',
                                  layout = widgets.Layout(width='300px'))

load_model_type_drop = widgets.interactive(load_show_models, 
                                target = load_target_drop,
                                mtype = load_type_drop, 
                                layout = widgets.Layout(width='700px'))

load_models = widgets.SelectMultiple(selected_labels = selectedModels, 
                           layout = widgets.Layout(width='700px'))

# Test dropdown
load_test_dd = widgets.Dropdown(options=[k for k in records.readings.keys()], 
                        layout=widgets.Layout(width='400px'),
                        description = 'Test')

load_models_interact = widgets.interactive(load_selectModels,
                                     selected_model = load_models,
                                     model_source= load_test_dd,
                                     layout = widgets.Layout(width='700px'))

load_min_date = widgets.Text(description='Start date:', 
                         layout=widgets.Layout(width='330px'))
load_max_date = widgets.Text(description='End date:', 
                         layout=widgets.Layout(width='330px'))



load_test_drop = widgets.interactive(load_show_devices, 
                                Source=load_test_dd, 
                                layout=widgets.Layout(width='400px'))

# Device dropdown
load_device = widgets.Dropdown(layout=widgets.Layout(width='200px'),
                        description = 'Device')

load_device_drop = widgets.interactive(load_show_dates, 
                                Source=load_device, 
                                layout=widgets.Layout(width='400px'))

# Sensor dropdown
load_result_text = widgets.Text(layout = widgets.Layout(width='300px'),
                               description = 'Result name')

load_calculateButton = widgets.Button(description='Predict channel')
load_calculateButton.on_click(load_calculateModel)
load_device_box = widgets.HBox([load_test_drop, load_device])
plot_result = widgets.Checkbox(value=True, 
                                     description='Plot Result', 
                                     disabled=False, 
                                     layout=widgets.Layout(width='300px'))
calculate_channel_box = widgets.HBox([load_result_text, load_calculateButton, plot_result])
display(load_model_type_drop)
display(load_models)

load_B = widgets.Button(description='Load Model')
load_B.on_click(loadModel)

buttonBox = widgets.VBox([load_B, load_device_box, load_min_date, load_max_date, calculate_channel_box])
display(buttonBox)
display(out_model_load)